# Laboratorio 1 - Procesamiento Distribuido de Datos

## Información del Estudiante

**Nombre:** Daniel Jose Plazas Cortes <br>
**Código:** A00400085 <br>

## Cargar Información

In [24]:
!pip install pandas multiprocessing time numpy

  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [25]:
import gdown

file_id = "1foHt1ncXXkqD5_WmTZkIfc6G6sQIuU9D"
url = f"https://drive.google.com/uc?id={file_id}"
output = "fraudDetection-activity.csv"

gdown.download(url, output, quiet=False)

fraud = pd.read_csv(output, index_col=0)

print(f"\nDimensiones del dataset: {fraud.shape[0]} filas x {fraud.shape[1]} columnas")
print(f"\nPrimeras 5 filas:")
fraud.head()

Downloading...
From (original): https://drive.google.com/uc?id=1foHt1ncXXkqD5_WmTZkIfc6G6sQIuU9D
From (redirected): https://drive.google.com/uc?id=1foHt1ncXXkqD5_WmTZkIfc6G6sQIuU9D&confirm=t&uuid=287cd34e-a44f-447f-aa07-a5848e995a5b
To: /content/fraudDetection-activity.csv
100%|██████████| 267M/267M [00:01<00:00, 184MB/s] 



Dimensiones del dataset: 1048575 filas x 24 columnas

Primeras 5 filas:


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,Unnamed: 23,6006
0,1/1/19 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,3495,"Psychologist, counselling",3/9/88,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,NaN,NaN
1,1/1/19 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,149,Special educational needs teacher,6/21/78,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN,NaN
2,1/1/19 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,4154,Nature conservation officer,1/19/62,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,NaN,NaN
3,1/1/19 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,1939,Patent attorney,1/12/67,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN,NaN
4,1/1/19 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,99,Dance movement psychotherapist,3/28/86,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,NaN,NaN


In [26]:
df = fraud[['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'lat', 'long', 'merch_lat', 'merch_long', 'is_fraud']]
df.head()
len(df)

1048575

## Procesamiento Secuencial

In [27]:
import time
import numpy as np

# Analisis secuencial

## Inicio tiempo
start_time = time.time()

## Total transacciones
total_transacciones = len(df)

## Total transacciones fraudulentas
total_transacciones_fraudulentas = len(df[df['is_fraud'] == 1])

## Monto promedio transacciones
monto_promedio_transacciones = df['amt'].mean()

## Monto promedio transacciones fraudulentas
monto_promedio_transacciones_fraudulentas = df[df['is_fraud'] == 1]['amt'].mean()

## Fin tiempo
end_time = time.time()

## Tiempo transcurrido
tiempo_transcurrido = end_time - start_time

print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")
print(f"Total de transacciones: {total_transacciones}")
print(f"Total de transacciones fradulentas: {total_transacciones_fraudulentas}")
print(f"Monto promedio transacciones: {monto_promedio_transacciones}")
print(f"Monto promedio transacciones fraudulentas: {monto_promedio_transacciones_fraudulentas}")


Tiempo transcurrido: 0.028704404830932617 segundos
Total de transacciones: 1048575
Total de transacciones fradulentas: 6006
Monto promedio transacciones: 70.27909503850466
Monto promedio transacciones fraudulentas: 530.5734915084914


## Procesamiento en Paralelo

In [28]:
import time
from multiprocessing import Pool

def calcular_estadisticas(df):
  ## Total transacciones
  total_transacciones = len(df)

  ## Total transacciones fraudulentas
  total_transacciones_fraudulentas = len(df[df['is_fraud'] == 1])

  ## Monto promedio transacciones
  monto_promedio_transacciones = df['amt'].sum()

  ## Monto promedio transacciones fraudulentas
  monto_promedio_transacciones_fraudulentas = df[df['is_fraud'] == 1]['amt'].sum()

  return total_transacciones, total_transacciones_fraudulentas, monto_promedio_transacciones, monto_promedio_transacciones_fraudulentas

# Analisis en paralelo

## Inicio tiempo
start_time = time.time()

## Dividir el dataset en multiples particiones
num_partitions = 10
partition_size = len(df) // num_partitions
partitions = [df[i * partition_size:(i + 1) * partition_size] for i in range(num_partitions)]

## Realizar los cálculos en paralelo para cada partición
with Pool(processes=num_partitions) as pool:
  results = pool.map(calcular_estadisticas, partitions)

## Consolidad resultados
total_transacciones = sum(result[0] for result in results)
total_transacciones_fraudulentas = sum(result[1] for result in results)
monto_promedio_transacciones = sum(result[2] for result in results) / len(df)
monto_promedio_transacciones_fraudulentas = sum(result[3] for result in results) / len(df[df['is_fraud'] == 1])

## Fin tiempo
end_time = time.time()

## Tiempo transcurrido
tiempo_transcurrido = end_time - start_time

print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")
print(f"Total de transacciones: {total_transacciones}")
print(f"Total de transacciones fradulentas: {total_transacciones_fraudulentas}")
print(f"Monto promedio transacciones: {monto_promedio_transacciones}")
print(f"Monto promedio transacciones fraudulentas: {monto_promedio_transacciones_fraudulentas}")

Tiempo transcurrido: 1.1685304641723633 segundos
Total de transacciones: 1048570
Total de transacciones fradulentas: 6006
Monto promedio transacciones: 70.27887422454283
Monto promedio transacciones fraudulentas: 530.5734915084917


## Tabla comparativa

| Métrica                                   | Secuencial           | Paralelo          |
| ----------------------------------------- | -------------------- | ----------------- |
| Tiempo transcurrido (s)                   | 0.017733335494995117 | 1.1685304641723633|
| Total de transacciones                    | 1,048,575            | 1,048,570         |
| Total de transacciones fraudulentas       | 6,006                | 6,006             |
| Monto promedio transacciones              | 70.27909503850466    | 70.27887422454283 |
| Monto promedio transacciones fraudulentas | 530.5734915084914    | 530.5734915084917 |


# Conclusiones

1) Los resultados mostraron que la ejecución paralela tardó más que la ejecución secuencial. Esto ocurrió porque el dataset era pequeño y el tiempo necesario para dividir los datos, crear hilos/procesos, coordinarlos y unir los resultados (overhead) fue mayor que el tiempo requerido para procesarlos secuencialmente.

4) Estos resultados se deben principalmente al costo de administración del paralelismo: creación de hilos, sincronización, comunicación entre procesos y acceso concurrente a memoria. El paralelismo local además está limitado por la cantidad de núcleos del procesador, la memoria disponible y la contención de recursos del sistema, lo que impide obtener mejoras significativas cuando la carga de trabajo es pequeña.

5) Si el dataset creciera a varios millones de registros, el paralelismo comenzaría a mostrar ventajas claras frente al procesamiento secuencial, ya que múltiples núcleos podrían procesar particiones grandes de datos simultáneamente. Sin embargo, podrían aparecer problemas como saturación de memoria RAM, mayor competencia por el ancho de banda de memoria y el límite físico del número de núcleos disponibles, lo que eventualmente podría requerir el uso de procesamiento distribuido en lugar de solo paralelismo local.